<a href="https://colab.research.google.com/github/Mangai2024/Mangai2024/blob/main/Multi_PDF_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain-community langchain-text-splitters faiss-cpu sentence-transformers pypdf transformers accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
#Step 1
!mkdir data



mkdir: cannot create directory ‘data’: File exists


In [ ]:
import os

os.listdir("data")


['diabetes_health.pdf',
 'machine_learning.pdf',
 'human_body.pdf',
 'solar_system.pdf']

In [ ]:
#Step 2 load all pdfs
from langchain_community.document_loaders import  PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("data")
documents = loader.load()

In [ ]:
#Step 3  Split into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap = 80
)
chunks = splitter.split_documents(documents)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(chunks, embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

hf = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def rag_answer(question):
    docs = vectorstore.similarity_search(question, k=3)
    context = "\n\n".join([d.page_content for d in docs])
    prompt = f"Text:\n{context}\n\nQuestion: {question}"
    return hf(prompt, max_new_tokens=150)[0]["generated_text"]

print(rag_answer("What is diabetes?"))


diabetes is a chronic condition where the body cannot properly process blood sugar (glucose). It occurs due to lack of insulin or the body's inability to use insulin effectively.


In [32]:
questions = [
    "What are the main systems in the human body?",
    "What does the brain do?",
    "Explain the nervous system.",
    "What are the planets in the solar system?",
    "What is diabetes?"
]

for q in questions:
    print("\nQ:", q)
    print("A:", rag_answer(q))



Q: What are the main systems in the human body?
A: circulatory system, respiratory system, digestive system, nervous system, muscular system, and skeletal system

Q: What does the brain do?
A: manages movement, thoughts, memory, and sensations

Q: Explain the nervous system.
A: The brain is the control center of the nervous system. It manages movement, thoughts, memory, and sensations. The digestive system breaks down food to provide energy. Muscles help in movement, and bones provide structure and protect organs. The human body is made up of several systems that work together to keep us alive. Major systems include the circulatory system, respiratory system, digestive system, nervous system, muscular system, and skeletal system. The circulatory system includes the heart and blood vessels, which pump blood throughout the body. The respiratory system includes the lungs, which help us breathe and exchange oxygen and carbon dioxide. The brain is the control center of the nervous system. 